# Analyzing CIA Factbook Data Using SQL

In this project we will use SQL to analyze some of the data in the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/). The purpose is to show how data stored in a database can be analyzed with SQL.

The CIA World Factbook provides various data points about 267 world entities. Below are some examples of these data points:
* Demographics
* Enonomy
* Transportation
* Energy
* Communications
* Government
* Military
* Geography
* History

The database file in this project contains a selection of these data points.

### Navigation

[(1) Connect to Database](#Connect-to-Database-Stored-Locally)

[(2) List Tables](#List-Tables-Currently-In-The-Database)

[(3) Inspect a Table](#Inspect-the-'facts'-Table)

[(4) Calculate Summary Statistics (MIN, MAX, SUM, AVG)](#Calculate-Summary-Statistics-For-Select-Fields)

[(5) Filter Data Dynamically by Using a Subquery](#View-Country-Names-With-Minimum-and-Maximum-Population)

[(6) Calculate Summary Statistics With a Condition](#Calculate-Summary-Statistics-For-Select-Fields-With-Filtering)

[(7) Answer An Additional Question About Data](#Answer-Questions-About-Data)

### Connect to Database Stored Locally

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

[To Top](#Navigation)

### List Tables Currently In The Database

In [2]:
%%sql

SELECT *
  FROM sqlite_master
 WHERE type = 'table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


[To Top](#Navigation)

### Inspect the 'facts' Table

In [3]:
%%sql

SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


This preview looks clean. The column headers are styled properly per database conventions

[To Top](#Navigation)

### Calculate Summary Statistics For Select Fields 

In [4]:
%%sql

SELECT MIN(population),
       MAX(population),
       MIN(population_growth),
       MAX(population_growth)
  FROM facts;

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


Minimum population is 0, which is either an error or is an entity that is not an inhabited area.
Maximum population is over 7 billion, which appears to be the total population of the world.
Minimum and maximum population growth seem plausible.

[To Top](#Navigation)

### View Country Names With Minimum and Maximum Population

In [5]:
%%sql

SELECT name, population
  FROM facts
 WHERE population == (SELECT MIN(population)
                        FROM facts);

Done.


name,population
Antarctica,0


In [6]:
%%sql

SELECT name, population
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts);

Done.


name,population
World,7256490011


This query confirms our suspicion, that not all entities in the database are countries. Some are continents or the entire world.

[To Top](#Navigation)

### Calculate Summary Statistics For Select Fields With Filtering

In [7]:
%%sql

SELECT MIN(population),
       MAX(population),
       MIN(population_growth),
       MAX(population_growth)
  FROM facts
 WHERE population <> (SELECT MAX(population) /* filter out maximum population */
                        FROM facts); 

Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [8]:
%%sql

SELECT AVG(population), AVG(area)
  FROM facts
 WHERE population <> (SELECT MAX(population) /* filter out maximum population */
                        FROM facts);

Done.


AVG(population),AVG(area)
32242666.56846473,582949.8523206752


[To Top](#Navigation)

### Answer Questions About Data

See list of countries with above average population and below average area (densely populated countries)

In [9]:
%%sql

SELECT name
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts)
   AND area       < (SELECT AVG(area)
                       FROM facts);
           

Done.


name
Bangladesh
Germany
Japan
Philippines
Thailand
United Kingdom
Vietnam
